In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df_movies=pd.read_csv("tmdb_5000_movies.csv")
df_credits=pd.read_csv("tmdb_5000_credits.csv")

In [4]:
df_movies = df_movies.merge(df_credits,on="title")
df_movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
df_movies.homepage[0]

'http://www.avatarmovie.com/'

In [6]:
df_movies2 = df_movies[["genres","overview","title","keywords","cast","crew","id"]]
df_movies2.dropna(inplace=True)
df_movies2.head()

,genres,overview,title,keywords,cast,crew,id
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",19995
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",285
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",206647
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",49026
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...",John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",49529


In [7]:
import ast
def convert_data(obj):
    data = []
    
    new_obj = ast.literal_eval(obj)
    for i in new_obj:
        data.append(i["name"])
        
    return data


In [8]:
def convert_data2(obj):
    new_obj = ast.literal_eval(obj)
    new_list = []
    for i in new_obj:
        if i["job"] == "Director":
            new_list.append(i["name"])
            return new_list
        
convert_data2(df_movies2["crew"][0])

['James Cameron']

In [9]:
df_movies2["genres"] = df_movies2["genres"].apply(convert_data)
df_movies2["cast"] = df_movies2["cast"].apply(convert_data)
df_movies2["keywords"] = df_movies2["keywords"].apply(convert_data)
df_movies2["crew"] = df_movies2["crew"].apply(convert_data2)
df_movies2["cast"] = df_movies2["cast"].apply(lambda x: x[:3])

In [10]:
df_movies2["overview"] = df_movies2["overview"].apply(lambda x:x.split())

In [11]:
df_movies2["genres"] = df_movies2["genres"].apply(lambda x: [i.replace(" ", "") for i in x])

In [12]:
df_movies2.dropna(inplace=True)
df_movies2.isnull().sum()

genres      0
overview    0
title       0
keywords    0
cast        0
crew        0
id          0
dtype: int64

In [13]:
df_movies2["keywords"] = df_movies2["keywords"].apply(lambda x: [i.replace(" ", "") for i in x])
df_movies2["cast"] = df_movies2["cast"].apply(lambda x: [i.replace(" ", "") for i in x])
df_movies2["crew"] = df_movies2["crew"].apply(lambda x: [i.replace(" ", "") for i in x])

In [14]:
final_df = df_movies2[["title","id"]]

In [15]:
final_df["tags"]=df_movies2["overview"] + df_movies2["keywords"] + df_movies2["cast"] + df_movies2["crew"] 

In [16]:
final_df["tags"] = final_df["tags"].apply(lambda x: " ".join(x))

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words="english")

In [19]:
final_df.to_csv('finaldf.csv', index=False)

In [20]:
vectors = cv.fit_transform(final_df["tags"]).toarray()

In [24]:
vectors.shape

(4776, 5000)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
similiarity = cosine_similarity(vectors)

In [25]:
similiarity.shape

(4776, 4776)

In [27]:
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Save the similarity matrix to a .pkl file
with open("similarity_matrix4.pkl", "wb") as file:
    pickle.dump(similiarity, file)

print("Similarity matrix saved to 'similarity_matrix.pkl'")

Similarity matrix saved to 'similarity_matrix.pkl'


In [23]:
def recommend(movie):
    recommed_movies = []
    movie_id = final_df.index[final_df['title'] == movie].tolist()[0]
    movie_sim = similiarity[movie_id]
    #sorted_numbers = sorted(movie_sim, reverse=True)
    sorted_numbers_with_indices = sorted(enumerate(movie_sim), key=lambda x: x[1], reverse=True)

    # Extract the sorted values and their corresponding indices
    sorted_indices = [index for index, value in sorted_numbers_with_indices][1:6]
    sorted_values = [value for index, value in sorted_numbers_with_indices][1:6]
    
    #print(sorted_indices,sorted_values)

    for i in sorted_indices:
        recommed_movies.append(final_df["title"][i])

    return recommed_movies

In [30]:
recommend("Spectre")

['Never Say Never Again',
 'Quantum of Solace',
 'Skyfall',
 'Safe Haven',
 'Fiddler on the Roof']

In [1]:
from flask import Flask, render_template, request
import requests

app = Flask(__name__)

In [2]:
# Example function to get movie poster and recommendations (mocked)
def get_movie_data(movie_name):
    # Mock data for demonstration
    # In a real application, replace this with API calls or a database query
    movie_data = {
        'The Matrix': {
            'poster': 'https://upload.wikimedia.org/wikipedia/en/b/ba/Poster_-_The_Matrix_Reloaded.jpg',
            'recommendations': [
                'Inception',
                'The Terminator',
                'Blade Runner',
                'Total Recall',
                'Dark City'
            ]
        },
        'Inception': {
            'poster': 'https://example.com/inception-poster.jpg',
            'recommendations': [
                'Interstellar',
                'Memento',
                'Shutter Island',
                'The Prestige',
                'The Matrix'
            ]
        }
        # Add more movies and their data as needed
    }
    return movie_data.get(movie_name, {'poster': None, 'recommendations': []})

In [3]:
@app.route('/', methods=['GET', 'POST'])
def index():
    main_movie_poster = None
    recommended_posters = []

    if request.method == 'POST':
        movie_name = request.form.get('movie_name')
        if movie_name:
            movie_data = get_movie_data(movie_name)
            main_movie_poster = movie_data['poster']
            recommendations = movie_data['recommendations']

            # Fetch posters for recommended movies
            recommended_posters = [get_movie_data(movie)['poster'] for movie in recommendations]

    return render_template('index.html', main_movie_poster=main_movie_poster, recommended_posters=recommended_posters)

In [5]:
if __name__ == '__main__':
    app.run(debug=True,host='0.0.0.0',port=5000) 

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.20.10.4:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [2]:
!curl --request GET url https://api.themoviedb.org/3/movie/19995/images header 'accept: application/json'

{"status_code":7,"status_message":"Invalid API key: You must be granted a valid key.","success":false}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl: (6) Could not resolve host: url
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   103    0   103    0     0     88      0 --:--:--  0:00:01 --:--:--    88
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0   

In [19]:
import requests

url = "https://api.themoviedb.org/3/movie/{}/images?api_key=fb47a9a0e7b454d24998042f495d0fe6".format{movie_id}
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)
image_path = "https://image.tmdb.org/t/p/w500"+response_json["poster_path"]

{"backdrops":[{"aspect_ratio":1.778,"height":2160,"iso_639_1":null,"file_path":"/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg","vote_average":6.512,"vote_count":36,"width":3840},{"aspect_ratio":1.778,"height":720,"iso_639_1":null,"file_path":"/ruziOM4OlILvyrOdChvvFqy4Ggw.jpg","vote_average":5.322,"vote_count":5,"width":1280},{"aspect_ratio":1.778,"height":2160,"iso_639_1":null,"file_path":"/tvaFREoQ7ssrPcwfz7Xbj2A7B2t.jpg","vote_average":5.318,"vote_count":3,"width":3840},{"aspect_ratio":1.778,"height":2160,"iso_639_1":"en","file_path":"/Antz70VlRI2yU3iwYxyYUknzfz9.jpg","vote_average":5.312,"vote_count":1,"width":3840},{"aspect_ratio":1.778,"height":2160,"iso_639_1":"en","file_path":"/ws8Rcopga4U9znbPaDZTY3LZLK3.jpg","vote_average":5.312,"vote_count":1,"width":3840},{"aspect_ratio":1.778,"height":2160,"iso_639_1":null,"file_path":"/yXsY0j90fwbfK4QVAFuQxH5LQzn.jpg","vote_average":5.258,"vote_count":6,"width":3840},{"aspect_ratio":1.778,"height":2160,"iso_639_1":null,"file_path":"/qCv5kKfYRAofZGxYCPW

In [31]:
import requests
import json
url = "https://api.themoviedb.org/3/movie/19995?api_key=fb47a9a0e7b454d24998042f495d0fe6&language=en-US"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)
response_json = json.loads(response.text)


In [32]:
response_json

{'adult': False,
 'backdrop_path': '/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg',
 'belongs_to_collection': {'id': 87096,
  'name': 'Avatar Collection',
  'poster_path': '/uO2yU3QiGHvVp0L5e5IatTVRkYk.jpg',
  'backdrop_path': '/gxnvX9kF7RRUQYvB52dMLPgeJkt.jpg'},
 'budget': 237000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 14, 'name': 'Fantasy'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'https://www.avatar.com/movies/avatar',
 'id': 19995,
 'imdb_id': 'tt0499549',
 'origin_country': ['US'],
 'original_language': 'en',
 'original_title': 'Avatar',
 'overview': 'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.',
 'popularity': 146.263,
 'poster_path': '/kyeqWdyUXW608qlYkRqosgbbJyK.jpg',
 'production_companies': [{'id': 444,
   'logo_path': None,
   'name': 'Dune Entertainment',
   'origin_country': 'US'},
  {'id': 

In [24]:
response_json["poster_path"]

KeyError: 'poster_path'

In [34]:

image_path

'https://image.tmdb.org/t/p/w500/kyeqWdyUXW608qlYkRqosgbbJyK.jpg'